In [1]:
#Kenneth Dandrow / CS-340 / Module 7 Project Two / 2/23/25

from jupyter_dash import JupyterDash

# Import necessary Dash components
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import dash_leaflet as dl
import plotly.express as px
import pandas as pd
import base64

# Import the AnimalShelter class from CRUD.py
from CRUD import AnimalShelter

# MongoDB authentication details (FIXME parts filled in)
username = "aacuser"
password = "snhu1234"

# Connect to MongoDB via CRUD module
db = AnimalShelter(username, password)

# Retrieve all animal data from MongoDB
df = pd.DataFrame.from_records(db.read({}))

# Remove the '_id' column to prevent errors in DataTable
if "_id" in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Initialize JupyterDash app
app = JupyterDash(__name__)

# Load Grazioso Salvare Logo (FIXME fixed)
image_filename = 'grazioso_logo.png'  # Make sure this file is in the correct directory
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

# Dashboard layout with logo, filters, and interactive elements
app.layout = html.Div([
    html.Center(html.B(html.H1('Grazioso Salvare Animal Shelter Dashboard by Kenneth Dandrow'))),
    html.Img(src=f'data:image/png;base64,{encoded_image}', style={'width': '200px'}),
    html.Hr(),

    # Interactive filtering options (FIXME fixed: Adding radio buttons for rescue type selection)
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'}
        ],
        value=None,
        labelStyle={'display': 'block'}
    ),
    html.Hr(),

    # Data Table with filtering enabled (FIXME fixed: Making it interactive)
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        filter_action="native",
        sort_action="native",
        row_selectable="single"
    ),
    html.Br(),
    html.Hr(),

    # Visualization area for charts and maps (FIXME fixed: Adding breed distribution and geolocation map)
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

# Callback to update the data table based on filter selection
@app.callback(
    Output('datatable-id', 'data'),
    Input('filter-type', 'value')
)
def update_dashboard(filter_type):
    filtered_df = df.copy()

    if filter_type:
        if filter_type == "Water Rescue":
            filtered_df = filtered_df[
                (filtered_df['breed'].isin(["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"])) &
                (filtered_df['sex_upon_outcome'] == "Intact Female") &
                (filtered_df['age_upon_outcome_in_weeks'].between(26, 156))
            ]
        elif filter_type == "Mountain or Wilderness Rescue":
            filtered_df = filtered_df[
                (filtered_df['breed'].isin(["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"])) &
                (filtered_df['sex_upon_outcome'] == "Intact Male") &
                (filtered_df['age_upon_outcome_in_weeks'].between(26, 156))
            ]
        elif filter_type == "Disaster or Individual Tracking":
            filtered_df = filtered_df[
                (filtered_df['breed'].isin(["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"])) &
                (filtered_df['sex_upon_outcome'] == "Intact Male") &
                (filtered_df['age_upon_outcome_in_weeks'].between(20, 300))
            ]

    return filtered_df.to_dict('records')  

# Callback to generate a breed distribution chart (FIXME fixed: Adding a pie chart)
@app.callback(
    Output('graph-id', "children"),
    Input('datatable-id', "derived_virtual_data")
)
def update_graph(viewData):
    if viewData is None or len(viewData) == 0:
        return "No data available"
    
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names='breed', title='Breed Distribution')

    return dcc.Graph(figure=fig)

# Callback to update the geolocation map (FIXME fixed: Adding map integration)
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if not viewData or index is None:
        return "No selection made"
    
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0

    return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
        dl.TileLayer(id="base-layer-id"),
        dl.Marker(position=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']], children=[
            dl.Tooltip(dff.iloc[row]['breed']),
            dl.Popup([html.H1("Animal Name"), html.P(dff.iloc[row]['name'])])
        ])
    ])

# Run JupyterDash app inline
app.run_server(debug=True)


Dash app running on http://127.0.0.1:27918/
